In [1]:
import onnxruntime as ort
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
import os

In [2]:
def test_onnx_model(image_path):
    # Label mapping
    labels = ['Bags', 'Bottomwear', 'Dress', 'Headwear', 'Shoes', 'Topwear', 'Watches']
    
    # Load and initialize ONNX model
    session = ort.InferenceSession("fashion_model.onnx")
    
    # Prepare image preprocessing
    transform = transforms.Compose([
        transforms.Resize((177, 177)),
        transforms.ToTensor(),
    ])
    
    # Load and preprocess image
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)  # Add batch dimension
    input_data = image.numpy()
    
    # Get input name
    input_name = session.get_inputs()[0].name
    
    # Run inference
    outputs = session.run(None, {input_name: input_data})
    
    # Convert outputs to probabilities
    probabilities = softmax(outputs[0][0])
    
    # Print results
    print(f"\nResults for image: {os.path.basename(image_path)}")
    print("-" * 50)
    for label, prob in zip(labels, probabilities):
        percentage = prob * 100
        print(f"{label:<15} : {percentage:.2f}%")

def softmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / exp_x.sum()

In [14]:
test_images = [
    "../Dataset/Shoes/shoes5811.jpg", #Cari direktori gambar yang mau di tes
]

for image_path in test_images:
    test_onnx_model(image_path)


Results for image: shoes5811.jpg
--------------------------------------------------
Bags            : 0.54%
Bottomwear      : 0.60%
Dress           : 58.17%
Headwear        : 0.65%
Shoes           : 0.56%
Topwear         : 39.44%
Watches         : 0.05%
